# Potentiel des méthodes de deep learning pour cartographier l’occupation du sol de La Réunion à partir d’images de télédétection

### Etape 1 : Création de la base de donnée 
Les dfférentes section de ce NoteBook permettent de créer une base de donnée pour entrainer et valider le réseau de neurone. <br>
Ce script permet d'obtenir : <br>
<ul>
<li>Un dossier contenant des sous dossier de 1000 imagettes chacun </li>
<li>Un fichier "vérité terrain" de type csv avec le chemin des imagettes, leurs classes et leurs labels </li>
</ul>


In [4]:
from osgeo import gdal

from osgeo import ogr

from osgeo import osr
#from gdalconst import *

from osgeo import gdalconst
import os

import numpy 
import csv

from datetime import datetime 
print(str(datetime.now()))

import random

2021-08-06 10:51:33.217770


>Le dossier ou seront crée les imagettes est défini ci dessous. <br>
Attention ! Il faut que le dossier n'existe pas au préalable, la fonction se charge de le créer. 

In [2]:
path = "Dataset_50x50_Niveau2_v9/"
os.mkdir(path)

FileExistsError: [WinError 183] Impossible de créer un fichier déjà existant: 'Dataset_50x50_Niveau2_v9/'

>src = image en Geotiff à découper <br>
bd = Vérité terrain (masque pré-traité sur Qgis) en .shp

In [10]:
src = 'Data_final/20201105_mosaic_pms_8bits.tif'
bd = 'Phase2/masque_projete_2020.shp'


In [ ]:
src2 = 'Data_final/2018_pléiades_run_pms_8bits.tif'
bd2 = 'Phase2/masque_finale_2018.shp'

>Résolution donc largeur et longueur exprimé en m

In [11]:
PixelRes= 0.5 

>Le "datafile_path" est le nom du fichier csv contenant les informations sur le chemin des imagettes, leurs classe en int et leur labels en string

In [12]:
RasterFormat ='GTiff'
VectorFormat='ESRI Shapefile'
datafile_path = 'Dataset_50x50_Niveau2_v9.csv'

In [13]:
# Open datasets
Raster =gdal.Open(src, gdal.GA_ReadOnly)
Projection = Raster.GetProjectionRef()

AttributeError: 'NoneType' object has no attribute 'GetProjectionRef'

In [ ]:
Raster2 =gdal.Open(src2, gdal.GA_ReadOnly)
Projection2 = Raster2.GetProjectionRef()

<b> Pour choisir le niveau de classification il faut changer la ligne " writer.writerow" avec les codes et niveaux correspondant. </b>

In [3]:
VectorDriver = ogr.GetDriverByName(VectorFormat)
VectorDataset = VectorDriver.Open(bd,0)# 0=Read-only, 1=Read-Write
layer = VectorDataset.GetLayer()
FeatureCount = layer.GetFeatureCount()

field_names=[field.name for field in layer.schema]
print(field_names)

NameError: name 'VectorFormat' is not defined

In [ ]:
VectorDriver2 = ogr.GetDriverByName(VectorFormat)
VectorDataset2 = VectorDriver2.Open(bd2,0)# 0=Read-only, 1=Read-Write
layer2 = VectorDataset2.GetLayer()
FeatureCount2 = layer2.GetFeatureCount()

field_names2=[field.name for field in layer2.schema]
print(field_names2)

In [76]:
# Iterate through the shapefile features
Count = 0
i =0

with open(datafile_path, 'w') as f: 
    writer = csv.writer(f)
    writer.writerow(['Image FileName',' Class', 'Label'])
    
    for feature in layer:
        if i==0 or i%1000==0:
            Count+=1
            os.mkdir(path+str(Count))
        i+=1
        geom = feature.GetGeometryRef() 
        minX, maxX, minY, maxY = geom.GetEnvelope() # Get bounding box of the shapefile feature
                #OutTileName = str(feature.GetField("id")+'.SomeTileName.tif'
            
        OutTileName = str(path)+str(Count)+'/'+str(feature.GetField("id"))+'.tif'
        
        gdal.Warp(OutTileName, Raster, format=RasterFormat, outputType = gdal.gdalconst.GDT_Byte, outputBounds=[minX, minY, maxX, maxY], xRes=PixelRes, yRes=PixelRes, dstSRS='EPSG:2975', resampleAlg=None, options=['COMPRESS=NONE'])
               
        writer.writerow([str(OutTileName), str(feature.GetField("Code4")),str(feature.GetField("Niveau2")), ])    
       
    for feature in layer2:
        if i==0 or i%1000==0:
            Count+=1
            os.mkdir(path+str(Count))
        i+=1
        geom = feature.GetGeometryRef() 
        minX, maxX, minY, maxY = geom.GetEnvelope() # Get bounding box of the shapefile feature
                #OutTileName = str(feature.GetField("id")+'.SomeTileName.tif'
        OutTileName = str(path)+str(Count)+'/'+str(feature.GetField("id"))+'.tif'
        gdal.Warp(OutTileName, Raster2, format=RasterFormat, outputType = gdal.gdalconst.GDT_Byte, outputBounds=[minX, minY, maxX, maxY], xRes=PixelRes, yRes=PixelRes, dstSRS='EPSG:2975', resampleAlg=None, options=['COMPRESS=NONE'])
                # output = gdal.Open(OutTileName, gdal.GA_ReadOnly)
                #OutTile = None # Close dataset
        writer.writerow([str(OutTileName), str(feature.GetField("Code2")),str(feature.GetField("Niveau2")), ])
        

        
   
        
        
    

In [77]:
# Close datasets
Raster = None
VectorDataset.Destroy()


Done.
2021-07-13 19:56:15.762019


In [ ]:
Raster2 = None
VectorDataset2.Destroy()

In [ ]:
print("Done.")
print(str(datetime.now()))